### Youtube Sentence Pos Tag Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification
threshold_num = 1

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def column_word_list(df, column):
    word_list = []
    for i in df[f"{column}"].dropna():
        words = i.split(",")
        for j in words:
            word = j.strip()
            word_list.append(word)
    return word_list

In [ ]:
def column_word(df, column_list):
    '''
    column_word(df_pos_tag_group, df_pos_tag_group.columns) \n
    column_word(df_pos_tag_group, ["col_1","col_2"])
    '''
    word_list_all = []
    for i in column_list:
        word_list = column_word_list(df, f"{i}")
        word_list_all.append(word_list)
    return word_list_all

In [ ]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    if any([True if i>count_condition else False for i in count_list]):
        return "Fail"
    else:
        return "Pass"

    #return any([True if i>count_condition else False for i in count_list])  

#### Part 1

In [ ]:
df_pos_tag_group = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/\
Data/{lang_folder.capitalize()}/{file}.xlsx", sheet_name=f"{sheets}")
df_pos_tag_group

In [ ]:
df_youtube_sent_select = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
2-Word Group In Youtube Sentence/Turkish English/Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
#for i in df_youtube_sent_select["search_string"]:
for i in range(len(df_youtube_sent_select)):
    #words = word_tokenize(i)
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    words = word_tokenize(search)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([search,start,end,sent,id,id_url,list_var])

In [ ]:
df_youtube_sent_class = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","classification"])
df_youtube_sent_class

In [ ]:
df_youtube_sent_class["condition"] = df_youtube_sent_class["classification"].apply(lambda x: classification_condition(x, threshold_num))
df_youtube_sent_class

In [ ]:
df_youtube_sent_class.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_All.xlsx", index=False)

In [ ]:
df_youtube_sent_class_select = df_youtube_sent_class[df_youtube_sent_class["condition"] == "Pass"]
df_youtube_sent_class_select.reset_index(drop=True, inplace=True)
df_youtube_sent_class_select

In [ ]:
df_youtube_sent_class_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_Select.xlsx", index=False)

#### Part 2

In [ ]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

In [ ]:
df_youtube_sent_select = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
2-Word Group In Youtube Sentence/Turkish English/Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

In [ ]:
df_youtube_sent_select["word_index_list"] = df_youtube_sent_select["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_select

In [ ]:
df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [ ]:
df_youtube_sent_select = df_youtube_sent_select[df_youtube_sent_select["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

In [ ]:
df_youtube_sent_select.info()

In [ ]:
result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + [x for x in word_index]

    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())

    if any([True if i>2 else False for i in count_list]):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])
    

In [ ]:
df_youtube_sent_select.loc[5,"word_index_list"]

In [ ]:
word_index = df_youtube_sent_select.loc[5,"word_index_list"]

In [ ]:
set_index = set(index_list)
set_all_index = set([x for x in range(0,200)])

In [ ]:
set_all_index.difference(set_index)

In [ ]:
Counter(index_list)

In [ ]:
df_test = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_test

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Custom Pattern

In [ ]:
def classification_order(classification_list, target_pattern_list):
    '''
    classification_order(classification_list, target_pattern_list)
    classification_list and target_pattern_list are list
    '''
    if [x for x in classification_list] == [y for y in target_pattern_list]:
        return "pattern is equal"
    else:
        return "pattern in not equal"

In [ ]:
target_pattern = [7,1,4,1,2,8,7,3,4,1,2,8,7,8,8]

In [ ]:
df_youtube_sent_class["pattern_result"] = df_youtube_sent_class["classification"].apply(lambda x: classification_order(x, target_pattern))
df_youtube_sent_class

In [ ]:
df_youtube_sent_class[df_youtube_sent_class["test"] == "pattern_result"]

In [ ]:
df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Custom_Pattern.xlsx", index=False)

In [ ]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [ ]:
df_youtube_sent_select["word_index_list"] = df_youtube_sent_select["search_string"].apply(lambda x: )

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

In [ ]:
word_index_dict = {key:val for (key, val) in zip(df_word_select["word"],df_word_select["index"])}
#word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word_select["word"])}

In [ ]:
word_index_dict["bu"]

In [ ]:
column_word_list(df_pos_tag_group, "col_1")

In [ ]:
column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
df_test = df_youtube_sent_select.head(2)
df_test

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
for i in df_test["search_string"]:
    words = word_tokenize(i)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([i,list_var])

In [ ]:
result_list

In [ ]:
pd.DataFrame(result_list, columns=["sentence","classification"])

In [ ]:
len(column_word_list_all)

In [ ]:
from collections import Counter

In [ ]:
result_list[0][1]

In [ ]:
Counter(result_list[0][1])

In [ ]:
any(Counter(result_list[0][1]).values())

In [ ]:
list_test = list(Counter(result_list[0][1]).values())
list_test

In [ ]:
any([True if i>3 else False for i in list_test])

In [ ]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    return any([True if i>count_condition else False for i in count_list])   

In [ ]:
list_test2 = result_list[0][1]

In [ ]:
classification_condition(list_test2, 4)